In [ ]:
import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
import spacy
import math
import os
import statsmodels.api as sm

from src.utils import *
from src.data_preprocessing import *
from src.data_processing import *
from src.feature_selection import *
from src.modelisation_arcticle_1 import *
from src.modelisation_arcticle_2 import *

os.chdir('/Users/ismailakrout/Desktop/python/NLP_statapp')

pd.set_option("display.max_columns", None)


## avec les articles 

In [ ]:
df_Guardian = pd.read_csv('00. input/newSplit_topics_bigfive_theGuardian_with_sentibert_post2010.csv')
df_DE = pd.read_csv('00. input/new_bigfive_DailyExpress_with_sentibert_post2010.csv')

Il faut d'abord effectuer un traitement similaire df_modelisation articles des journdf_modelisation

In [ ]:
def treat_1(df, party):
  df = df[['author', 'fulltext']]
  if party == 'Guardian' :
    df['party'] = 'Lab'
  if party == 'DE' :
    df['party'] = 'Con'
  df.rename(columns=
          {'author' : 'Speaker', 'fulltext':'text'}, inplace=True)
  return df

In [ ]:
def treat_2(df_1,df_2):
  df = pd.concat([df_1, df_2])
  df.dropna().reset_index(drop=True)
  return df

In [ ]:
def treat_3(df):
  df['text'] = df['text'].apply(clean, args=(stop_words,))
  return df 

In [ ]:
df_Guardian = treat_1(df_Guardian, 'Guardian')

In [ ]:
df_DE = treat_1(df_DE, 'DE')

In [ ]:
df_newspaper = treat_2(df_Guardian, df_DE)

In [ ]:
df_newspaper = df_newspaper.sample(frac=1).reset_index(drop=True)
df_newspaper = df_newspaper.head(1000)

In [ ]:
df_newspaper = treat_3(df_newspaper)

NameError: name 'stop_words' is not defined

In [ ]:
df_newspaper_treated = df_newspaper

In [ ]:
df_newspaper_treated = df_newspaper_treated.groupby(by=['Speaker', 'party']).sum().reset_index()

In [ ]:
df_freqs_Con = count_freqs(df_newspaper_treated, 'Con')
df_freqs_Lab = count_freqs(df_newspaper_treated, 'Lab')

In [ ]:
df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)

In [ ]:
list_of_words = selected_words(df_freqs)

In [ ]:
df_newspaper_treated = construct_df_reg(df_newspaper_treated, df_freqs, list_of_words)

In [ ]:
df_newspaper_treated['Unnamed: 0'] = 0

In [ ]:
LO_treatment(df_newspaper_treated)

In [ ]:
def treat_fonction(df_1, df_2):
  df_newspaper = treat_2(df_1, df_2)
  df_newspaper = df_newspaper.sample(frac=1).reset_index(drop=True)
  df_newspaper = df_newspaper.head(100)
  df_newspaper = treat_3(df_newspaper)
  df_newspaper_treated = df_newspaper
  df_newspaper_treated = df_newspaper_treated.groupby(by=['Speaker', 'party']).sum().reset_index()
  df_freqs_Con = count_freqs(df_newspaper_treated, 'Con')
  df_freqs_Lab = count_freqs(df_newspaper_treated, 'Lab')
  df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)
  list_of_words = selected_words(df_freqs)
  df_newspaper_treated = construct_df_reg(df_newspaper_treated, df_freqs, list_of_words)
  df_newspaper_treated['Unnamed: 0'] = 0
  return LO_treatment(df_newspaper_treated)

In [ ]:
treat_fonction(df_Guardian, df_DE)

# **Évolution de la polarisation dans le temps**

In [ ]:
df_Guardian = pd.read_csv('/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/data/TheGuardian/newSplit_topics_bigfive_theGuardian_with_sentibert_post2010.csv')
df_DE = pd.read_csv('/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/data/DailyExpress/new_bigfive_DailyExpress_with_sentibert_post2010.csv')

In [ ]:
def treat_1_date(df, party, year):
  df = df.loc[df['year'] == year]
  df = df[['author', 'fulltext']]
  if party == 'Guardian' :
    df['party'] = 'Lab'
  if party == 'DE' :
    df['party'] = 'Con'
  df.rename(columns=
          {'author' : 'Speaker', 'fulltext':'text'}, inplace=True)
  return df

In [ ]:
def treat_fonction(df_1, df_2):
  df_newspaper = treat_2(df_1, df_2)
  df_newspaper = df_newspaper.sample(frac=1).reset_index(drop=True)
  df_newspaper = df_newspaper.head(100)
  df_newspaper = treat_3(df_newspaper)
  df_newspaper_treated = df_newspaper
  df_newspaper_treated = df_newspaper_treated.groupby(by=['Speaker', 'party']).sum().reset_index()
  df_freqs_Con = count_freqs(df_newspaper_treated, 'Con')
  df_freqs_Lab = count_freqs(df_newspaper_treated, 'Lab')
  df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)
  list_of_words = selected_words(df_freqs)
  df_newspaper_treated = construct_df_reg(df_newspaper_treated, df_freqs, list_of_words)
  df_newspaper_treated['Unnamed: 0'] = 0
  print('SORTIE')
  return LO_treatment(df_newspaper_treated)

In [ ]:
df_1 = treat_1_date(df_Guardian, 'Guardian', 2020)
df_2 = treat_1_date(df_DE, 'DE', 2020)

In [ ]:
treat_fonction(df_1, df_2)

In [ ]:
evolution_of_pi = []

In [ ]:
for year in range(2010, 2023 ):
  df_a = treat_1_date(df_Guardian, 'Guardian', year)
  df_b = treat_1_date(df_DE, 'DE', year)
  pi = treat_fonction(df_a, df_b)
  evolution_of_pi.append(pi)
  print(evolution_of_pi, 'BOUCLE')

In [ ]:
evolution_of_pi

In [ ]:
plt.plot([i for i in range(2010, 2023)], evolution_of_pi)
plt.show()

# **Partisanship des mots**

In [ ]:
df_Guardian = pd.read_csv('/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/data/TheGuardian/newSplit_topics_bigfive_theGuardian_with_sentibert_post2010.csv')
df_DE = pd.read_csv('/content/drive/MyDrive/Statapp_TextDigit/Stage_2022/data/DailyExpress/new_bigfive_DailyExpress_with_sentibert_post2010.csv')

In [ ]:
df_Guardian = treat_1(df_Guardian, 'Guardian')
df_DE = treat_1(df_DE, 'DE')

In [ ]:
df_Guardian = treat_1(df_Guardian, 'Guardian')
df_DE = treat_1(df_DE, 'DE')

In [ ]:
def treat_table(df_1, df_2):
  df_newspaper = treat_2(df_1, df_2)
  df_newspaper = df_newspaper.sample(frac=1).reset_index(drop=True)
  df_newspaper = df_newspaper.head(100)
  df_newspaper = treat_3(df_newspaper)
  df_newspaper_treated = df_newspaper
  df_newspaper_treated = df_newspaper_treated.groupby(by=['Speaker', 'party']).sum().reset_index()
  df_freqs_Con = count_freqs(df_newspaper_treated, 'Con')
  df_freqs_Lab = count_freqs(df_newspaper_treated, 'Lab')
  df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)
  list_of_words = selected_words(df_freqs)
  df_newspaper_treated = construct_df_reg(df_newspaper_treated, df_freqs, list_of_words)
  df_newspaper_treated['Unnamed: 0'] = 0
  return LO_treatment_table(df_newspaper_treated)

In [ ]:
table_Con, table_Lab = treat_table(df_Guardian, df_DE)

In [ ]:
table_Con.dropna().reset_index(drop=True)
table_Lab.dropna().reset_index(drop=True)